In [68]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [69]:
df_origin = pd.read_csv('./visualization/dataset/titanic3.csv')
df = df_origin.copy()
df1 = df.copy() # 수정한 데이터 적용할곳
df.head()
# pclass 티켓등급
# survived 생존여부
# name 이름
# sex 성별
# sibsp 탑승한 형제, 배우자의 수
# parch 탑승한 부모, 자녀의 수
# ticket 티켓 번호
# fare 요금
# cabin 객실 번호
# embarked 기항지 위치
# boat 탈출한 보트가 있다면 boat 번호
# body 사망자 확인 번호
# home.dest 고향/목적지

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [71]:
print(df['home.dest'].value_counts(),'\n')
print(df.loc[df['home.dest']=='New York, NY'].mean())
# 절반도 겹치지 않으나 가장많은 값인 교향이 뉴욕인 사람의 생존률이 62퍼센트이므로 뉴욕만 보존하기로 결정
df1['home.dest'] = np.where(df1['home.dest']=='New York, NY',1,0)  # 뉴욕만 1로 바꿔줌
df1['home.dest'].value_counts()

New York, NY                                    64
London                                          14
Montreal, PQ                                    10
Paris, France                                    9
Cornwall / Akron, OH                             9
                                                ..
Chelsea, London                                  1
Harrow-on-the-Hill, Middlesex                    1
Copenhagen, Denmark                              1
Guernsey / Montclair, NJ and/or Toledo, Ohio     1
Antwerp, Belgium / Stanton, OH                   1
Name: home.dest, Length: 369, dtype: int64 

pclass       1.187500
survived     0.625000
age         38.063636
sibsp        0.500000
parch        0.187500
fare        62.976041
body        63.400000
dtype: float64


0    1245
1      64
Name: home.dest, dtype: int64

In [72]:
# body는 사망자 확인번호이므로 body가 존재한다면 무조건 사망자
print(df.body.sort_values())
# 1부터 있으므로 NaN값을 0으로 만들고 0 이상을 1로 바꾸기
df1.body.replace(np.nan,0, inplace = True)
df1.body.loc[df.body>0] =1
df1.body.value_counts()

1263     1.0
1096     4.0
1153     7.0
741      9.0
1294    14.0
        ... 
1302     NaN
1303     NaN
1305     NaN
1307     NaN
1308     NaN
Name: body, Length: 1309, dtype: float64


0.0    1188
1.0     121
Name: body, dtype: int64

In [73]:
df_em = df[['embarked','survived']].groupby('embarked')
print(df_em.mean())
# embarked 가 C 인경우 생존률이 높으므로 유의미한 데이터라고 판단
df2 = pd.get_dummies(df1.embarked) # 원핫인코딩으로 분류
df1 = df1.join(df2)
df1.drop('embarked',axis = 1 , inplace = True) # embarked 열 삭제
df1.head()

          survived
embarked          
C         0.555556
Q         0.357724
S         0.332604


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,boat,body,home.dest,C,Q,S
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,2,0.0,0,0,0,1
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,11,0.0,0,0,0,1
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,NaN,0.0,0,0,0,1
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,NaN,1.0,0,0,0,1
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,NaN,0.0,0,0,0,1


In [74]:
# boat는 보트를 탑승했다는 뜻이므로 보트에 탔다면 생존률이 높을것이라 예상
# boat에 값이 있다면 1 없다면 0으로 
print(df.boat.value_counts().sum())
df1.boat.fillna(0,inplace = True)
df1.boat=np.where(df1.boat==0,0,1)
df1.boat.value_counts()

486


0    823
1    486
Name: boat, dtype: int64

In [75]:
# boat를 그룹으로 묶어서 확인해본 결과 생존률이 98%로 매우 높은것을 확인
df_boat= df1.groupby('boat')
df_boat.mean()

,pclass,survived,age,sibsp,parch,fare,body,home.dest,C,Q,S
boat,,,,,,,,,,,
0,2.503038,0.027947,30.504897,0.520049,0.325638,23.241605,0.147023,0.030377,0.147023,0.103281,0.749696
1,1.942387,0.981481,28.940264,0.462963,0.485597,50.300180,0.000000,0.080247,0.306584,0.078189,0.611111


In [76]:
print(df.cabin.value_counts())
# cabin의 경우 겹치는것도 많이 없고 양 자체가 적으므로 삭제
df1.drop('cabin',axis=1,inplace = True)
df1.head()

C23 C25 C27        6
G6                 5
B57 B59 B63 B66    5
F4                 4
F33                4
                  ..
C132               1
E60                1
B52 B54 B56        1
C49                1
F38                1
Name: cabin, Length: 186, dtype: int64


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,boat,body,home.dest,C,Q,S
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,1,0.0,0,0,0,1
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,1,0.0,0,0,0,1
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,0,0.0,0,0,0,1
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,0,1.0,0,0,0,1
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,0,0.0,0,0,0,1


In [77]:
print(df1.fare.value_counts(),'\n')
# 너무 광범위 하므로 범주를 설정
print(df1.fare.isnull().sum(),'\n') # NaN 값이 존재하므로 처리 필요
print(df.loc[df.fare.isnull()]) # fare 값은 결국 pclass와 관련이 있다고 생각

8.0500     60
13.0000    59
7.7500     55
26.0000    50
7.8958     49
           ..
15.0500     1
9.6875      1
15.5792     1
12.0000     1
7.8750      1
Name: fare, Length: 281, dtype: int64 

1 

      pclass  survived                name   sex   age  sibsp  parch ticket  \
1225       3         0  Storey, Mr. Thomas  male  60.5      0      0   3701   

      fare cabin embarked boat   body home.dest  
1225   NaN   NaN        S  NaN  261.0       NaN  


In [78]:
# 확인해보니 등급별로 fare차이가 있으므로 NaN값을 pclass 3의 fare평균값으로 대체하기로 결정
print(df.loc[df.pclass == 1,'fare'].mean(),'\n')
print(df.loc[df.pclass == 2,'fare'].mean(),'\n')
print(df.loc[df.pclass == 3,'fare'].mean(),'\n')
df1.fare.replace(np.nan , df1.loc[df.pclass==3,'fare'].mean() , inplace = True)
df1.fare.isnull().sum()

87.50899164086687 

21.1791963898917 

13.302888700564957 



0

In [79]:
df.loc[:,['fare','pclass']].groupby('pclass').describe()
# pclass 2,3 의 최대값이 73이므로 75를 이상을 범주로 묶고 그 아래를 나누기로 결정

fare                                                                 
        count       mean        std  min      25%      50%       75%       max
pclass                                                                        
1       323.0  87.508992  80.447178  0.0  30.6958  60.0000  107.6625  512.3292
2       277.0  21.179196  13.607122  0.0  13.0000  15.0458   26.0000   73.5000
3       708.0  13.302889  11.494358  0.0   7.7500   8.0500   15.2458   69.5500

In [80]:
np.histogram(df1.fare,5)

(array([1225,   46,   34,    0,    4], dtype=int64),
 array([  0.     , 102.46584, 204.93168, 307.39752, 409.86336, 512.3292 ]))

In [81]:
fare_name = ['0','5','10','15','20','25','30','35','40','45','50','55','60','65','70','75']
fare_div = [0 , 5, 10,15, 20,25, 30,35, 40,45,50,55, 60, 65,70, 75 , 512.3292]
df1['fare_cut'] = pd.cut(x = df1['fare'], #데이터 배열
                               bins = fare_div,    #경계값 리스트
                               labels = fare_name,     #구간명
                               include_lowest = True)  # 첫 경계값 포함여부(구간의 하위값)

In [82]:
df1.groupby('fare_cut').describe()
#괜찮다고 판단하여 이대로 해보기로 결정

pclass                                              survived  \
          count      mean       std  min  25%  50%  75%  max    count   
fare_cut                                                                
0          20.0  1.900000  0.852242  1.0  1.0  2.0  3.0  3.0     20.0   
5         471.0  2.997877  0.046078  2.0  3.0  3.0  3.0  3.0    471.0   
10        183.0  2.289617  0.454829  2.0  2.0  2.0  3.0  3.0    183.0   
15         79.0  2.810127  0.394707  2.0  3.0  3.0  3.0  3.0     79.0   
20         69.0  2.666667  0.474858  2.0  2.0  3.0  3.0  3.0     69.0   
25        144.0  1.659722  0.681044  1.0  1.0  2.0  2.0  3.0    144.0   
30         48.0  2.020833  0.887012  1.0  1.0  2.0  3.0  3.0     48.0   
35         31.0  1.903226  0.746317  1.0  1.0  2.0  2.0  3.0     31.0   
40          8.0  1.500000  0.534522  1.0  1.0  1.5  2.0  2.0      8.0   
45         16.0  2.000000  1.032796  1.0  1.0  2.0  3.0  3.0     16.0   
50         28.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0     28.0   
55         28.0  1.571429  0.920087  1.0  1.0  1.0  3.0  3.0     28.0   
60         13.0  1.384615  0.506370  1.0  1.0  1.0  2.0  2.0     13.0   
65         15.0  2.466667  0.915475  1.0  2.0  3.0  3.0  3.0     15.0   
70         11.0  1.636364  0.504525  1.0  1.0  2.0  2.0  2.0     11.0   
75        145.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0    145.0   

                    ...    Q           S                                     \
              mean  ...  75%  max  count      mean       std  min  25%  50%   
fare_cut            ...                                                       
0         0.150000  ...  0.0  0.0   20.0  0.950000  0.223607  0.0  1.0  1.0   
5         0.227176  ...  0.0  1.0  471.0  0.681529  0.466379  0.0  0.0  1.0   
10        0.327869  ...  0.0  1.0  183.0  0.819672  0.385515  0.0  1.0  1.0   
15        0.493671  ...  0.0  1.0   79.0  0.569620  0.498293  0.0  0.0  1.0   
20        0.420290  ...  0.0  1.0   69.0  0.797101  0.405104  0.0  1.0  1.0   
25        0.458333  ...  0.0  1.0  144.0  0.784722  0.412450  0.0  1.0  1.0   
30        0.312500  ...  0.0  0.0   48.0  0.812500  0.394443  0.0  1.0  1.0   
35        0.580645  ...  0.0  0.0   31.0  0.806452  0.401610  0.0  1.0  1.0   
40        0.375000  ...  0.0  0.0    8.0  0.375000  0.517549  0.0  0.0  0.0   
45        0.125000  ...  0.0  0.0   16.0  0.812500  0.403113  0.0  1.0  1.0   
50        0.607143  ...  0.0  0.0   28.0  0.964286  0.188982  0.0  1.0  1.0   
55        0.821429  ...  0.0  0.0   28.0  0.500000  0.509175  0.0  0.0  0.5   
60        0.615385  ...  0.0  0.0   13.0  0.538462  0.518875  0.0  0.0  1.0   
65        0.200000  ...  0.0  0.0   15.0  0.866667  0.351866  0.0  1.0  1.0   
70        0.181818  ...  0.0  0.0   11.0  0.818182  0.404520  0.0  1.0  1.0   
75        0.724138  ...  0.0  1.0  145.0  0.420690  0.495381  0.0  0.0  0.0   

                    
          75%  max  
fare_cut            
0         1.0  1.0  
5         1.0  1.0  
10        1.0  1.0  
15        1.0  1.0  
20        1.0  1.0  
25        1.0  1.0  
30        1.0  1.0  
35        1.0  1.0  
40        1.0  1.0  
45        1.0  1.0  
50        1.0  1.0  
55        1.0  1.0  
60        1.0  1.0  
65        1.0  1.0  
70        1.0  1.0  
75        1.0  1.0  

[16 rows x 96 columns]

In [38]:
# fare 행 삭제
df1.drop('fare',axis = 1, inplace = True)

In [39]:
df1.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,boat,body,home.dest,C,Q,S,fare_cut
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,1,0.0,0,0,0,1,over 75
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,1,0.0,0,0,0,1,over 75
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,0,0.0,0,0,0,1,over 75
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,0,1.0,0,0,0,1,over 75
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,0,0.0,0,0,0,1,over 75


,pclass,survived,age,sibsp,parch,boat,body,home.dest,C,Q,S,family
pclass,1.000000,-0.312469,-0.408106,0.060832,0.018322,-0.323428,-0.049382,-0.299785,-0.269658,0.230491,0.096335,0.050027
survived,-0.312469,1.000000,-0.055512,-0.027825,0.082660,0.948190,-0.250897,0.113408,0.182123,-0.016071,-0.154558,0.026876
age,-0.408106,-0.055512,1.000000,-0.243699,-0.150917,-0.053176,0.159975,0.133804,0.085777,-0.019458,-0.075972,-0.240229
sibsp,0.060832,-0.027825,-0.243699,1.000000,0.373587,-0.026488,-0.046510,0.000250,-0.048396,-0.048678,0.075198,0.861952
parch,0.018322,0.082660,-0.150917,0.373587,1.000000,0.089321,-0.038374,-0.051761,-0.008635,-0.100943,0.073258,0.792296
boat,-0.323428,0.948190,-0.053176,-0.026488,0.089321,1.000000,-0.245246,0.111734,0.190527,-0.041548,-0.145869,0.031397
body,-0.049382,-0.250897,0.159975,-0.046510,-0.038374,-0.245246,1.000000,-0.011203,0.000274,-0.039500,0.025930,-0.051566
home.dest,-0.299785,0.113408,0.133804,0.000250,-0.051761,0.111734,-0.011203,1.000000,0.173349,-0.048733,-0.121073,-0.028126
C,-0.269658,0.182123,0.085777,-0.048396,-0.008635,0.190527,0.000274,0.173349,1.000000,-0.164166,-0.775441,-0.036553
Q,0.230491,-0.016071,-0.019458,-0.048678,-0.100943,-0.041548,-0.039500,-0.048733,-0.164166,1.000000,-0.489874,-0.087190


In [8]:
df = pd.read_pickle('./visualization/dataset/tdf.pkl')
df.head()

,survived,fare_cat,age_cat,family,female,male,town_C,town_Q,town_S
0,1,1,0,0,1,0,0,0,1
1,1,1,4,3,0,1,0,0,1
2,0,1,4,3,1,0,0,0,1
3,0,1,0,3,0,1,0,0,1
4,0,1,0,3,1,0,0,0,1


In [11]:
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y_df = df['survived'] # 종속변수
X_df = df.drop('survived',axis=1) # 독립변수

# 학습용 데이터와 평가용 데이터를 8:2 혹은 7:3으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_df,y_df, test_size = 0.2, random_state=11) # 8:2로 , 랜덤고정

print(X_train.shape)
print(X_test.shape)

(1047, 8)
(262, 8)


In [15]:
# 모델 학습 및 평가
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test,rf_pred).round(2)

lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)
accuracy_lr = accuracy_score(y_test,lr_pred).round(2)

print(f'rf 정확도 : {accuracy_rf}, lr 정확도 : {accuracy_lr}')

rf 정확도 : 0.77, lr 정확도 : 0.81
